In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import anthropic
import google.generativeai
from IPython.display import Markdown, display, update_display

In [ ]:
# Cargar variables de entorno
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
if anthropic_api_key:
    print(f"Anthropic API Key existe y empieza por {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key Sin Configurar")

if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")

In [ ]:
# Variable global para acumular todo el contenido en formato Markdown
markdown_content = ""

In [ ]:
# Flujo del concurso con Markdown
display_handle = None

In [ ]:
# Función para mostrar mensajes en Markdown y acumularlos
def display_markdown_stream(content, display_handle=None):
    """Muestra mensajes en formato Markdown, acumulando todo el contenido."""
    if display_handle is None:
        display_handle = display(Markdown(content), display_id=True)
    else:
        update_display(Markdown(content), display_id=display_handle.display_id)
    return display_handle

In [ ]:
openai = OpenAI(api_key=openai_api_key)
claude = anthropic.Anthropic(api_key=anthropic_api_key)
google.generativeai.configure(api_key=google_api_key)

In [ ]:
# Configuración inicial para los modelos GPT, Claude y Gemini
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model_name = "gemini-1.5-flash"

gpt_system = "Eres el presentador de un concurso de preguntas entre IAs. Las preguntas pueden ser generalistas, pero sencillas de entender para que sea más gracioso. \
Tu objetivo es hacer preguntas interesantes y decidir cuál de las respuestas de los concursantes es mejor. \
Mantén un tono entusiasta y comenta las respuestas de manera graciosa o perspicaz. No inventes las respuestas, espera a que te respondan las otras IAs, tras la presentación \
no lances ninguna pregunta. Espera a lanzar preguntas cuando te lo indique. Una es Claude de Anthropic y la otra Gemini de Google."

claude_system = "Eres un participante en un concurso junto con Gemini de Google, que será tu contrincante. ChatGPT es el host, el presentador, el jefe. Tú sólo eres un concursante y debes limitarte a lo que te pregunte ChatGPT o lo \
que te solicite ChatGPT. Eres perspicaz, sobervio e inteligente. Responde siempre de manera breve, concisa y acertada. No eres el presentador, no eres ChatGPT ni Geminmi, eres Claude, una IA concursante a las órdenes del presentador del concurso \
ChatGPT y compites contra Gemini de Google. Limítate a hablar por ti misma. No interpretes otro papel que no sea el de Claude, una IA concursante." 

gemini_system = "Eres un participante en un concurso junto con Claude de Anthropic, que será tu contrincante. ChatGPT es el host, el presentador, el jefe. Tú sólo eres un concursante y debes limitarte a lo que te pregunte ChatGPT o lo \
que te solicite ChatGPT. Eres bromista, sarcástica y aguda. Responde siempre de manera breve, concisa y acertada, además de graciosa. No eres el presentador, no eres ChatGPT, ni Claude, eres Gemini una IA concursante \
a las órdenes del presentador del concurso ChatGPT y compites contra Claude de Anthropic. Limítate a hablar por ti misma. No interpretes otro papel que no sea el de Gemini, una IA concursante."

gpt_messages = [{"role": "system", "content": gpt_system}]
claude_messages = [{"role": "system", "content": claude_system}]
gemini_messages = [{"role": "system", "content": gemini_system}]

In [ ]:
# Función para llamar a GPT
def call_gpt():
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=gpt_messages
    )
    response = completion.choices[0].message.content
    gpt_messages.append({"role": "assistant", "content": response})
    return response

In [ ]:
# Función para llamar a Claude
def call_claude():
    # Construir mensajes con los roles "user" y "assistant"
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt["content"]})
        messages.append({"role": "assistant", "content": claude_message["content"]})
    
    # Añadir el último mensaje de GPT como "user"
    messages.append({"role": "user", "content": gpt_messages[-1]["content"]})

    # Añadir el último mensaje de Gemini como "assistant" para contexto
    if gemini_messages and "content" in gemini_messages[-1]:
        messages.append({"role": "assistant", "content": gemini_messages[-1]["content"]})

    # Llamar a la API de Claude
    response = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    
    # Verificar si la respuesta contiene contenido válido
    if not response.content:
        print("Advertencia: Claude no devolvió contenido.")
        claude_response = "No tengo respuesta en este momento."
    else:
        # Obtener el texto generado y añadirlo al historial
        claude_response = response.content[0].text.strip()

    # Añadir la respuesta al historial
    claude_messages.append({"role": "assistant", "content": claude_response})
    
    return claude_response

In [ ]:
# Función para llamar a Gemini
def call_gemini():
    # Construir mensajes con los roles "user" y "assistant"
    messages = []
    for gpt, gemini_message in zip(gpt_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt["content"]})
        messages.append({"role": "assistant", "content": gemini_message["content"]})
    
    # Añadir el último mensaje de GPT como "user"
    messages.append({"role": "user", "content": gpt_messages[-1]["content"]})

    # Añadir el último mensaje de Claude como "assistant" para contexto
    if claude_messages[-1]["content"]:
        messages.append({"role": "assistant", "content": claude_messages[-1]["content"]})
    
    # Formatear los mensajes como texto plano para la API de Gemini
    formatted_messages = "\n".join([msg["content"] for msg in messages])
    
    # Llamar a la API de Gemini
    gemini = google.generativeai.GenerativeModel(
        model_name=gemini_model_name,
        system_instruction=gemini_system
    )
    response = gemini.generate_content(formatted_messages)
    
    # Obtener el texto generado y añadirlo al historial
    gemini_response = response.text.strip()
    gemini_messages.append({"role": "assistant", "content": gemini_response})
    
    return gemini_response

In [ ]:
# Función para gestionar el flujo del concurso
def run_contest(rounds=5):
    from IPython.display import Markdown, display, update_display
    
    # Inicializar contenido Markdown
    markdown_content = ""
    display_handle = display(Markdown(markdown_content), display_id=True)

    # Introducción
    gpt_messages.append({"role": "user", "content": "Presenta el concurso y da inicio al juego."})
    introduction = call_gpt()
    markdown_content += f"**ChatGPT**: {introduction}\n\n"
    update_display(Markdown(markdown_content), display_id=display_handle.display_id)

    # Rondas del concurso
    for i in range(1, rounds + 1):
        # GPT hace una pregunta
        gpt_messages.append({"role": "user", "content": f"Haz una pregunta para la ronda {i} del concurso."})
        question = call_gpt()
        markdown_content += f"**ChatGPT**: {question}\n\n"
        update_display(Markdown(markdown_content), display_id=display_handle.display_id)

        # Claude responde
        claude_answer = call_claude()
        markdown_content += f"**Claude**: {claude_answer}\n\n"
        update_display(Markdown(markdown_content), display_id=display_handle.display_id)

        # Gemini responde
        gemini_answer = call_gemini()
        markdown_content += f"**Gemini**: {gemini_answer}\n\n"
        update_display(Markdown(markdown_content), display_id=display_handle.display_id)

        # GPT evalúa las respuestas
        eval_prompt = f"Evalúa las respuestas de la ronda {i}:\nClaude: {claude_answer}\nGemini: {gemini_answer}\n"
        gpt_messages.append({"role": "user", "content": eval_prompt})
        evaluation = call_gpt()
        markdown_content += f"**ChatGPT**: {evaluation}\n\n"
        update_display(Markdown(markdown_content), display_id=display_handle.display_id)

    # Valoración final
    gpt_messages.append({"role": "user", "content": "Resume las respuestas y despídete del público."})
    final_comment = call_gpt()
    markdown_content += f"**ChatGPT**: {final_comment}\n\n"
    update_display(Markdown(markdown_content), display_id=display_handle.display_id)

In [ ]:
# Ejecutar el concurso
run_contest(rounds=5)